In [1]:
import torch, os, copy, time
from tqdm import tqdm
from torch import nn
from torch.utils.data import DataLoader, Dataset
from utils import *
import torch.nn.functional as F
from torch.autograd import Variable
from functools import partial

Epoch: 1 | Batch index: 0 | Batch size: 128
break minibatch for-loop
Epoch: 2 | Batch index: 0 | Batch size: 128
break minibatch for-loop


In [2]:
class logReg(torch.nn.Module):
    # logistic regression 
    def __init__(self, num_features, num_classes):
        super().__init__()
        self.num_classes = num_classes
        self.linear = torch.nn.Linear(num_features, num_classes)

    def forward(self, x):
        logits = self.linear(x.float())
        probas = torch.sigmoid(logits)
        return probas.type(torch.FloatTensor), logits

global_model = logReg(num_features=NUM_FEATURES, num_classes=2)
global_weights = global_model.state_dict()

In [3]:
def fairLoss(log_probas, target, sensitive, logits, penalty = 0.1, threshold = 0.01):
    # penalty term: covariance between sensitive attributes and deicision boundary
    # the n, n_k yet to be added!
    return nn.NLLLoss().to(DEVICE)(log_probas, target) + Variable((
        penalty * torch.sum(((sensitive - mean_sensitive) * (logits.T[0] * (2*target-1))) - threshold) ** 2
        ), requires_grad = True)

In [4]:
class DatasetSplit(Dataset):
    """An abstract Dataset class wrapped around Pytorch Dataset class.
    """

    def __init__(self, dataset, idxs):
        self.dataset = dataset
        self.idxs = [int(i) for i in idxs]

    def __len__(self):
        return len(self.idxs)

    def __getitem__(self, item):
        feature, label, sensitive = self.dataset[self.idxs[item]]
        return feature, label, sensitive


class LocalUpdate(object):
    def __init__(self, dataset, idxs, loss_func = None, penalty = 0.01, threshold = 0.1):
        self.trainloader, self.validloader, self.testloader = self.train_val_test(dataset, list(idxs))
        self.loss_func = loss_func
        # Default criterion set to NLL loss function
        if loss_func == None:
            self.criterion = nn.NLLLoss().to(DEVICE)
        elif loss_func == "fair penalty":
            self.criterion = partial(fairLoss, penalty = penalty, threshold = threshold)
            
    def train_val_test(self, dataset, idxs):
        """
        Returns train, validation and test dataloaders for a given dataset
        and user indexes.
        """
        # split indexes for train, validation, and test (80, 10, 10)
        idxs_train = idxs[:int(0.8*len(idxs))]
        idxs_val = idxs[int(0.8*len(idxs)):int(0.9*len(idxs))]
        idxs_test = idxs[int(0.9*len(idxs)):]

        trainloader = DataLoader(DatasetSplit(dataset, idxs_train),
                                 batch_size=BATCH_SIZE, shuffle=True)
        validloader = DataLoader(DatasetSplit(dataset, idxs_val),
                                 batch_size=int(len(idxs_val)/10), shuffle=False)
        testloader = DataLoader(DatasetSplit(dataset, idxs_test),
                                batch_size=int(len(idxs_test)/10), shuffle=False)
        return trainloader, validloader, testloader

    def update_weights(self, model, global_round):
        # Set mode to train model
        model.train()
        epoch_loss = []

        # Set optimizer for the local updates
        if OPTIMIZER == 'sgd':
            optimizer = torch.optim.SGD(model.parameters(), lr=LEARNING_RATE,
                                        momentum=0.5)
        elif OPTIMIZER == 'adam':
            optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE,
                                         weight_decay=1e-4)

        for i in range(LOCAL_EPOCHS):
            batch_loss = []
            for batch_idx, (features, labels, sensitive) in enumerate(self.trainloader):
                features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
                # we need to set the gradients to zero before starting to do backpropragation 
                # because PyTorch accumulates the gradients on subsequent backward passes. 
                # This is convenient while training RNNs
                model.zero_grad()
                
                loss = Variable(torch.Tensor([0]), requires_grad = True)
                log_probs, logits = model(features)
                if self.loss_func == None:
                    loss = self.criterion(log_probs, labels)
                elif self.loss_func == "fair penalty":
                    loss = self.criterion(log_probs, labels, sensitive, logits)
                loss.backward()
                optimizer.step()

                if batch_idx % 50 == 0:
                    print('| Global Round : {} | Local Epoch : {} | [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        global_round, i, batch_idx * len(features),
                        len(self.trainloader.dataset),
                        100. * batch_idx / len(self.trainloader), loss.item()))
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))

        # weight, loss
        return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

    def inference(self, model):
        """ 
        Returns the inference accuracy, 
                                loss, 
                                N(sensitive group, pos), 
                                N(non-sensitive group, pos), 
                                N(sensitive group),
                                N(non-sensitive group)
        """

        model.eval()
        loss, total, correct = 0.0, 0.0, 0.0
        sp, nsp, s, n = 0, 0, 0, 0
        for batch_idx, (features, labels, sensitive) in enumerate(self.testloader):
            features, labels = features.to(DEVICE), labels.to(DEVICE).type(torch.LongTensor)
            sensitive = sensitive.to(DEVICE)
            
            # Inference
            outputs, logits = model(features)

            # Prediction
            _, pred_labels = torch.max(outputs, 1)
            pred_labels = pred_labels.view(-1)
            bool_correct = torch.eq(pred_labels, labels)
            correct += torch.sum(bool_correct).item()
            total += len(labels)
            
            bool_sensitive = torch.eq(sensitive, torch.ones(len(labels)))
            s += torch.sum(bool_sensitive).item()
            n += torch.sum(torch.logical_not(bool_sensitive)).item()
            sp += torch.sum(torch.logical_and(bool_correct, bool_sensitive)).item()
            nsp += torch.sum(torch.logical_and(bool_correct, torch.logical_not(bool_sensitive))).item()
            
            if self.loss_func == None:
                batch_loss = self.criterion(outputs, labels)
            elif self.loss_func == "fair penalty":
                batch_loss = self.criterion(outputs, labels, sensitive, logits)
            loss += batch_loss.item()

        accuracy = correct/total
        return accuracy, loss, s, n, sp, nsp


def test_inference(model, test_dataset):
    """ Returns the test accuracy and loss.
    """

    model.eval()
    loss, total, correct = 0.0, 0.0, 0.0
    sp, nsp, s, n = 0, 0, 0, 0
    
    criterion = nn.NLLLoss().to(DEVICE)
    testloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                            shuffle=False)

    for batch_idx, (features, labels, sensitive) in enumerate(testloader):
        features = features.to(DEVICE)
        labels =  labels.to(DEVICE).type(torch.LongTensor)
        # Inference
        outputs, logits = model(features)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()

        # Prediction
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        bool_correct = torch.eq(pred_labels, labels)
        correct += torch.sum(bool_correct).item()
        total += len(labels)

        bool_sensitive = torch.eq(sensitive, torch.ones(len(labels)))
        s += torch.sum(bool_sensitive).item()
        n += torch.sum(torch.logical_not(bool_sensitive)).item()
        sp += torch.sum(torch.logical_and(bool_correct, bool_sensitive)).item()
        nsp += torch.sum(torch.logical_and(bool_correct, torch.logical_not(bool_sensitive))).item()

    accuracy = correct/total
    # |P(Group1, pos) - P(Group2, pos)| = |N(Group1, pos)/N(Group1) - N(Group2, pos)/N(Group2)|
    return accuracy, loss, abs(sp/s-nsp/n)

In [5]:
def train(loss_func = None, penalty = None, threshold = None):
    # Training
    train_loss, train_accuracy = [], []
    val_acc_list, net_list = [], []
    cv_loss, cv_acc = [], []
    print_every = 2
    val_loss_pre, counter = 0, 0
    start_time = time.time()

    def average_weights(w):
        """
        Returns the average of the weights.
        """
        w_avg = copy.deepcopy(w[0])
        for key in w_avg.keys():
            for i in range(1, len(w)):
                w_avg[key] += w[i][key]
            w_avg[key] = torch.div(w_avg[key], len(w))
        return w_avg

    for epoch in tqdm(range(NUM_EPOCHS)):
        local_weights, local_losses = [], []
        print(f'\n | Global Training Round : {epoch+1} |\n')

        global_model.train()
        m = max(int(FRAC * NUM_CLIENTS), 1) # the number of clients to be chosen in each epoch
        idxs_users = np.random.choice(range(NUM_CLIENTS), m, replace=False)

        for idx in idxs_users:
            local_model = LocalUpdate(dataset=train_dataset,
                                        idxs=clients_idx[idx], loss_func = loss_func, penalty = penalty, threshold = threshold)
            w, loss = local_model.update_weights(
                model=copy.deepcopy(global_model), global_round=epoch)
            local_weights.append(copy.deepcopy(w))
            local_losses.append(copy.deepcopy(loss))

        # update global weights
        global_weights = average_weights(local_weights)
        global_model.load_state_dict(global_weights)

        loss_avg = sum(local_losses) / len(local_losses)
        train_loss.append(loss_avg)

        # Calculate avg training accuracy over all users at every epoch
        list_acc, list_loss = [], []
        s, n, sp, nsp = 0, 0, 0, 0
        global_model.eval()
        for c in range(NUM_CLIENTS):
            local_model = LocalUpdate(dataset=train_dataset,
                                        idxs=clients_idx[idx], loss_func= loss_func, penalty = penalty, threshold = threshold)
            acc, loss, s_, n_, sp_, nsp_ = local_model.inference(model=global_model)
            list_acc.append(acc)
            list_loss.append(loss)
            s += s_
            n += n_
            sp += sp_
            nsp += nsp_
        train_accuracy.append(sum(list_acc)/len(list_acc))

        # print global training loss after every 'i' rounds
        if (epoch+1) % print_every == 0:
            print(f' \nAvg Training Stats after {epoch+1} global rounds:')
            print(f'Training Loss : {np.mean(np.array(train_loss))}')
            print('Train Accuracy: {:.2f}%'.format(100*train_accuracy[-1]))
            print('Train Risk Difference: {:.2f} \n'.format(abs(sp/s-nsp/n)))

    # Test inference after completion of training
    test_acc, test_loss, rd= test_inference(global_model, test_dataset)

    print(f' \n Results after {NUM_EPOCHS} global rounds of training:')
    print("|---- Avg Train Accuracy: {:.2f}%".format(100*train_accuracy[-1]))
    print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

    # Compute RD: risk difference - fairness metric
    # |P(Group1, pos) - P(Group2, pos)| = |N(Group1, pos)/N(Group1) - N(Group2, pos)/N(Group2)|
    print("|---- Test RD: {:.2f}".format(rd))

    print('\n Total Run Time: {0:0.4f}'.format(time.time()-start_time))

In [8]:
train("fair penalty", 100, 0.01)

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/9945 (0%)]	Loss: 1483446.625000
| Global Round : 0 | Local Epoch : 0 | [6400/9945 (64%)]	Loss: 1488485.500000
| Global Round : 0 | Local Epoch : 1 | [0/9945 (0%)]	Loss: 21002.060547
| Global Round : 0 | Local Epoch : 1 | [6400/9945 (64%)]	Loss: 910912.750000
| Global Round : 0 | Local Epoch : 2 | [0/9945 (0%)]	Loss: 177493.765625
| Global Round : 0 | Local Epoch : 2 | [6400/9945 (64%)]	Loss: 2417584.500000
| Global Round : 0 | Local Epoch : 3 | [0/9945 (0%)]	Loss: 191690.812500
| Global Round : 0 | Local Epoch : 3 | [6400/9945 (64%)]	Loss: 790100.250000
| Global Round : 0 | Local Epoch : 4 | [0/9945 (0%)]	Loss: 738826.812500
| Global Round : 0 | Local Epoch : 4 | [6400/9945 (64%)]	Loss: 1168737.250000
| Global Round : 0 | Local Epoch : 0 | [0/16103 (0%)]	Loss: 1595399.875000
| Global Round : 0 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: 1058552.875000
| Global Round : 0 | Local Epoch : 0 | [12800/16103 (79%)]	L

 20%|██        | 1/5 [00:03<00:15,  3.86s/it]


 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/9945 (0%)]	Loss: 340954.281250
| Global Round : 1 | Local Epoch : 0 | [6400/9945 (64%)]	Loss: 1560371.375000
| Global Round : 1 | Local Epoch : 1 | [0/9945 (0%)]	Loss: 762487.062500
| Global Round : 1 | Local Epoch : 1 | [6400/9945 (64%)]	Loss: 666319.375000
| Global Round : 1 | Local Epoch : 2 | [0/9945 (0%)]	Loss: 1123224.875000
| Global Round : 1 | Local Epoch : 2 | [6400/9945 (64%)]	Loss: 445818.281250
| Global Round : 1 | Local Epoch : 3 | [0/9945 (0%)]	Loss: 297466.625000
| Global Round : 1 | Local Epoch : 3 | [6400/9945 (64%)]	Loss: 1265456.625000
| Global Round : 1 | Local Epoch : 4 | [0/9945 (0%)]	Loss: 877713.187500
| Global Round : 1 | Local Epoch : 4 | [6400/9945 (64%)]	Loss: 557083.062500
| Global Round : 1 | Local Epoch : 0 | [0/16103 (0%)]	Loss: 1101378.125000
| Global Round : 1 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: 978984.375000
| Global Round : 1 | Local Epoch : 0 | [12800/16103 (79%)]	Lo

 40%|████      | 2/5 [00:07<00:11,  3.92s/it]

...


 | Global Training Round : 4 |

| Global Round : 3 | Local Epoch : 0 | [0/16103 (0%)]	Loss: 1832417.375000
| Global Round : 3 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: 3547347.750000
| Global Round : 3 | Local Epoch : 0 | [12800/16103 (79%)]	Loss: 349216.812500
| Global Round : 3 | Local Epoch : 1 | [0/16103 (0%)]	Loss: 734192.187500
| Global Round : 3 | Local Epoch : 1 | [6400/16103 (40%)]	Loss: 670339.812500
| Global Round : 3 | Local Epoch : 1 | [12800/16103 (79%)]	Loss: 1614150.000000
| Global Round : 3 | Local Epoch : 2 | [0/16103 (0%)]	Loss: 1892933.375000
| Global Round : 3 | Local Epoch : 2 | [6400/16103 (40%)]	Loss: 2083859.750000
| Global Round : 3 | Local Epoch : 2 | [12800/16103 (79%)]	Loss: 747347.187500
| Global Round : 3 | Local Epoch : 3 | [0/16103 (0%)]	Loss: 209722.625000
| Global Round : 3 | Local Epoch : 3 | [6400/16103 (40%)]	Loss: 1023798.125000
| Global Round : 3 | Local Epoch : 3 | [12800/16103 (79%)]	Loss: 410254.531250
| Global Round : 3 | Local Epoch : 4 

 80%|████████  | 4/5 [00:15<00:03,  3.89s/it]

| Global Round : 3 | Local Epoch : 4 | [6400/9945 (64%)]	Loss: 1986334.375000
 
Avg Training Stats after 4 global rounds:
Training Loss : 1076155.7835924276
Train Accuracy: 73.15%
Train Risk Difference: 0.20 


 | Global Training Round : 5 |

| Global Round : 4 | Local Epoch : 0 | [0/16103 (0%)]	Loss: 515493.781250
| Global Round : 4 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: 453020.875000
| Global Round : 4 | Local Epoch : 0 | [12800/16103 (79%)]	Loss: 741284.500000
| Global Round : 4 | Local Epoch : 1 | [0/16103 (0%)]	Loss: 898638.250000
| Global Round : 4 | Local Epoch : 1 | [6400/16103 (40%)]	Loss: 27296.525391
| Global Round : 4 | Local Epoch : 1 | [12800/16103 (79%)]	Loss: 2141947.750000
| Global Round : 4 | Local Epoch : 2 | [0/16103 (0%)]	Loss: 2004316.750000
| Global Round : 4 | Local Epoch : 2 | [6400/16103 (40%)]	Loss: 2283183.500000
| Global Round : 4 | Local Epoch : 2 | [12800/16103 (79%)]	Loss: 770984.187500
| Global Round : 4 | Local Epoch : 3 | [0/16103 (0%)]	Loss: 14

100%|██████████| 5/5 [00:19<00:00,  3.86s/it]


 
 Results after 5 global rounds of training:
|---- Avg Train Accuracy: 73.07%
|---- Test Accuracy: 76.39%
|---- Test RD: 0.19

 Total Run Time: 19.6859


In [7]:
train()

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |

| Global Round : 0 | Local Epoch : 0 | [0/9945 (0%)]	Loss: -0.999285
| Global Round : 0 | Local Epoch : 0 | [6400/9945 (64%)]	Loss: -0.999501
| Global Round : 0 | Local Epoch : 1 | [0/9945 (0%)]	Loss: -0.999621
| Global Round : 0 | Local Epoch : 1 | [6400/9945 (64%)]	Loss: -0.999673
| Global Round : 0 | Local Epoch : 2 | [0/9945 (0%)]	Loss: -0.999669
| Global Round : 0 | Local Epoch : 2 | [6400/9945 (64%)]	Loss: -0.999634
| Global Round : 0 | Local Epoch : 3 | [0/9945 (0%)]	Loss: -0.999636
| Global Round : 0 | Local Epoch : 3 | [6400/9945 (64%)]	Loss: -0.999679
| Global Round : 0 | Local Epoch : 4 | [0/9945 (0%)]	Loss: -0.999647
| Global Round : 0 | Local Epoch : 4 | [6400/9945 (64%)]	Loss: -0.999589
| Global Round : 0 | Local Epoch : 0 | [0/16103 (0%)]	Loss: -0.999735
| Global Round : 0 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: -0.999681
| Global Round : 0 | Local Epoch : 0 | [12800/16103 (79%)]	Loss: -0.999720
| Global Round : 0 | Local Epoch : 1 |

 20%|██        | 1/5 [00:03<00:15,  3.80s/it]


 | Global Training Round : 2 |

| Global Round : 1 | Local Epoch : 0 | [0/16103 (0%)]	Loss: -0.999697
| Global Round : 1 | Local Epoch : 0 | [6400/16103 (40%)]	Loss: -0.999673
| Global Round : 1 | Local Epoch : 0 | [12800/16103 (79%)]	Loss: -0.999690
| Global Round : 1 | Local Epoch : 1 | [0/16103 (0%)]	Loss: -0.999717
| Global Round : 1 | Local Epoch : 1 | [6400/16103 (40%)]	Loss: -0.999703
| Global Round : 1 | Local Epoch : 1 | [12800/16103 (79%)]	Loss: -0.999666
| Global Round : 1 | Local Epoch : 2 | [0/16103 (0%)]	Loss: -0.999768
| Global Round : 1 | Local Epoch : 2 | [6400/16103 (40%)]	Loss: -0.999764
| Global Round : 1 | Local Epoch : 2 | [12800/16103 (79%)]	Loss: -0.999712
| Global Round : 1 | Local Epoch : 3 | [0/16103 (0%)]	Loss: -0.999736
| Global Round : 1 | Local Epoch : 3 | [6400/16103 (40%)]	Loss: -0.999658
| Global Round : 1 | Local Epoch : 3 | [12800/16103 (79%)]	Loss: -0.999715
| Global Round : 1 | Local Epoch : 4 | [0/16103 (0%)]	Loss: -0.999725
| Global Round : 1 | 

 20%|██        | 1/5 [00:06<00:25,  6.42s/it]

| Global Round : 1 | Local Epoch : 4 | [12800/16103 (79%)]	Loss: -0.999672
| Global Round : 1 | Local Epoch : 0 | [0/9945 (0%)]	Loss: -0.999363


KeyboardInterrupt: 